In [31]:
import pandas as pd
import numpy as np
import quandl
from datetime import datetime, timedelta
import json


from dateutil.parser import parse
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold

In [3]:
# pull in cleaned dataset

data = pd.read_csv('data/combined_clean.csv', low_memory=False)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27492 entries, 0 to 27491
Data columns (total 45 columns):
Unnamed: 0              27492 non-null int64
ticker_symbol           27492 non-null object
co_name                 27492 non-null object
unique_earnings_code    27492 non-null object
factset_sector_num      27492 non-null float64
factset_ind_num         27492 non-null float64
calendar_qtr            27492 non-null object
fiscal_qtr              27492 non-null object
adtv_prev_month         27492 non-null object
report_date             27492 non-null object
eps_est                 27492 non-null object
eps_actual              27492 non-null object
surp_amt                27492 non-null object
rtn_t+3                 27492 non-null float64
mkt_t+3_rtn             27492 non-null float64
rel_t+3_rtn             27492 non-null float64
num_ests_qtr_end        27492 non-null float64
t-7_high_est            27492 non-null float64
t-7_low_est             27492 non-null float64
est_spread

In [41]:
report_dates = list(data['report_date']) 



In [42]:
new_dates = []

for dt in report_dates:
    new_dt = datetime.strptime(dt, '%m/%d/%Y')
    new_dates.append(dt)



In [54]:
new_dates[0]

'05/15/2014'

In [48]:
data['report_date'] = new_dates

In [49]:
type(data['report_date'])

pandas.core.series.Series

In [53]:
type(data['report_date'][0])

str

In [47]:
data.sort_values(by=['report_date'])

,Unnamed: 0,ticker_symbol,co_name,unique_earnings_code,factset_sector_num,factset_ind_num,calendar_qtr,fiscal_qtr,adtv_prev_month,report_date,...,si_days_1m_delta_F,si_days_3m_delta_F,si_pct_1m_delta_F,si_pct_3m_delta_F,3m_delta_an_rtg_F,pre_rpt_rtn_to_tgt_F,3m_delta_tgt_px_F,1m_est_delta_F,3m_est_delta_F,proj_2yr_rev_gr_F
21587,23325,CMC,Commercial Metals Company,CMC.4Q17,1200.0,1205.0,4Q17,2018/1F,34.1,01/03/2018,...,0.533070,0.181709,-0.024939,-0.099696,-0.28,-2.13,-5.39,-22.542,-35.172,3.610222
17014,18383,UNF,UniFirst Corporation,UNF.4Q16,3400.0,3450.0,4Q16,2017/1F,13.0,01/04/2017,...,0.000000,0.037642,-0.154043,-0.243324,0.07,-13.04,-7.60,0.952,-7.826,7.087855
22387,24197,RAD,Rite Aid Corporation,RAD.4Q17,3500.0,3510.0,4Q17,2017/3F,46.0,01/04/2018,...,0.000000,0.552878,-0.022833,0.150800,0.02,5.42,9.09,6.225,91.205,-3.374046
22382,24189,WBA,Walgreens Boots Alliance Inc,WBA.4Q17,3500.0,3510.0,4Q17,2018/1F,542.6,01/04/2018,...,0.000000,0.186893,-0.055397,0.114882,0.12,17.37,7.94,1.151,1.747,10.654890
22038,23811,MON,Monsanto Company,MON.4Q17,2200.0,2215.0,4Q17,2018/1F,312.3,01/04/2018,...,0.000000,0.159441,0.124182,0.419674,0.00,8.31,-0.49,-7.463,-6.478,13.648534
21845,23602,LW,"Lamb Weston Holdings, Inc.",LW.4Q17,2400.0,2410.0,4Q17,2017/2F,45.7,01/04/2018,...,0.000000,0.268574,0.369955,2.102440,0.00,-0.20,-6.05,-2.138,-2.138,11.082187
22124,23908,RPM,RPM International Inc.,RPM.4Q17,2200.0,2240.0,4Q17,2017/2F,39.9,01/04/2018,...,0.000000,0.195231,0.013443,-0.110706,-0.04,7.31,-0.72,-2.791,-4.603,9.996458
10979,11856,CMC,Commercial Metals Company,CMC.4Q15,1200.0,1205.0,4Q15,2016/1F,20.9,01/05/2016,...,0.000000,-0.207966,-0.109770,-0.447458,0.00,21.59,9.20,-29.575,-46.202,-27.761760
16923,18286,STZ,"Constellation Brands, Inc. Class A",STZ.4Q16,2400.0,2425.0,4Q16,2016/3F,284.0,01/05/2017,...,0.000000,-0.072079,0.038383,0.391582,-0.05,16.81,1.34,1.996,1.828,21.565086
16138,17431,MON,Monsanto Company,MON.4Q16,2200.0,2215.0,4Q16,2017/1F,220.6,01/05/2017,...,0.000000,0.750425,0.009475,-0.186841,-0.03,15.64,-1.11,-704.760,-117.096,-7.415412


In [58]:
def transform_report_dates(df):
    """Helper function to convert report_date column to a sortable format"""
    dates = list(df['report_date'])
    new_dates = []

    for d in dates:
        dt = datetime.strptime(d, '%m/%d/%Y')
        reformatted = str(dt)
        reformatted = reformatted[:10]
        new_dates.append(reformatted)
        
    df['report_date'] = new_dates    

In [56]:
transform_report_dates(data)

In [57]:
data.head()

,Unnamed: 0,ticker_symbol,co_name,unique_earnings_code,factset_sector_num,factset_ind_num,calendar_qtr,fiscal_qtr,adtv_prev_month,report_date,...,si_days_1m_delta_F,si_days_3m_delta_F,si_pct_1m_delta_F,si_pct_3m_delta_F,3m_delta_an_rtg_F,pre_rpt_rtn_to_tgt_F,3m_delta_tgt_px_F,1m_est_delta_F,3m_est_delta_F,proj_2yr_rev_gr_F
0,0,RAMP,"LiveRamp Holdings, Inc.",RAMP.1Q14,3300.0,3305.0,1Q14,2013/4F,14.4,2014-05-15,...,-0.144062,0.496136,0.057002,0.172538,0.15,33.83,4.44,1.266,7.527,-2.946171
1,2,TCO,"Taubman Centers, Inc.",TCO.1Q14,4800.0,4890.0,1Q14,2014/1F,37.3,2014-04-25,...,0.352543,0.223015,0.171640,0.150831,0.00,2.49,-0.10,-2.722,-3.706,-2.351877
2,5,SNX,SYNNEX Corporation,SNX.1Q14,3250.0,3265.0,1Q14,2014/1F,17.4,2014-04-04,...,0.000000,0.457639,0.069347,0.973769,-0.20,15.84,-5.39,1.198,-7.997,20.412552
3,7,AIV,Apartment Investment and Management Company Cl...,AIV.1Q14,4800.0,4890.0,1Q14,2014/1F,62.7,2014-05-02,...,0.582350,0.547657,-0.160877,-0.058043,0.07,2.13,-2.40,43.314,-3.597,-5.047765
4,10,IT,"Gartner, Inc.",IT.1Q14,3200.0,3205.0,1Q14,2014/1F,51.1,2014-05-01,...,0.015320,-0.107287,-0.218061,-0.025564,0.04,7.41,0.97,-0.270,-22.105,21.560045


### Experimenting with Quancha API 

In [ ]:
def add_vol_slope(df):
    """docstring"""
    
    # quandl authentication
    quandl.ApiConfig.api_key = "jUKZLy5xi7gGFf3sSF-r"
    
    

In [10]:
quandl.ApiConfig.api_key = "jUKZLy5xi7gGFf3sSF-r"

In [8]:
toy = data.iloc[:25,:]

In [9]:
toy

,Unnamed: 0,ticker_symbol,co_name,unique_earnings_code,factset_sector_num,factset_ind_num,calendar_qtr,fiscal_qtr,adtv_prev_month,report_date,...,si_days_1m_delta_F,si_days_3m_delta_F,si_pct_1m_delta_F,si_pct_3m_delta_F,3m_delta_an_rtg_F,pre_rpt_rtn_to_tgt_F,3m_delta_tgt_px_F,1m_est_delta_F,3m_est_delta_F,proj_2yr_rev_gr_F
0,0,RAMP,"LiveRamp Holdings, Inc.",RAMP.1Q14,3300.0,3305.0,1Q14,2013/4F,14.4,05/15/2014,...,-0.144062,0.496136,0.057002,0.172538,0.15,33.83,4.44,1.266,7.527,-2.946171
1,2,TCO,"Taubman Centers, Inc.",TCO.1Q14,4800.0,4890.0,1Q14,2014/1F,37.3,04/25/2014,...,0.352543,0.223015,0.171640,0.150831,0.00,2.49,-0.10,-2.722,-3.706,-2.351877
2,5,SNX,SYNNEX Corporation,SNX.1Q14,3250.0,3265.0,1Q14,2014/1F,17.4,04/04/2014,...,0.000000,0.457639,0.069347,0.973769,-0.20,15.84,-5.39,1.198,-7.997,20.412552
3,7,AIV,Apartment Investment and Management Company Cl...,AIV.1Q14,4800.0,4890.0,1Q14,2014/1F,62.7,05/02/2014,...,0.582350,0.547657,-0.160877,-0.058043,0.07,2.13,-2.40,43.314,-3.597,-5.047765
4,10,IT,"Gartner, Inc.",IT.1Q14,3200.0,3205.0,1Q14,2014/1F,51.1,05/01/2014,...,0.015320,-0.107287,-0.218061,-0.025564,0.04,7.41,0.97,-0.270,-22.105,21.560045
5,12,MPWR,"Monolithic Power Systems, Inc.",MPWR.1Q14,1300.0,1305.0,1Q14,2014/1F,23.0,04/25/2014,...,-0.216623,-0.166703,0.029595,-0.058553,-0.02,10.10,-13.44,-0.475,-6.250,26.303779
6,14,WETF,"WisdomTree Investments, Inc.",WETF.1Q14,4800.0,4845.0,1Q14,2014/1F,39.2,05/02/2014,...,-0.163667,0.697238,-0.043391,0.779130,-0.09,42.81,9.57,-3.379,-7.241,113.565568
7,15,SLG,SL Green Realty Corp.,SLG.1Q14,4800.0,4890.0,1Q14,2014/1F,58.5,04/24/2014,...,0.205259,0.479865,0.051846,0.014484,0.03,-0.41,-2.20,16.468,6.833,-3.442242
8,16,SKX,"Skechers U.S.A., Inc. Class A",SKX.1Q14,2400.0,2440.0,1Q14,2014/1F,41.2,04/23/2014,...,0.822023,-0.319613,-0.104418,-0.466693,0.21,12.92,-8.86,1.005,8.065,34.137421
9,17,BNNY,"Annie's, Inc.",BNNY.1Q14,2400.0,2410.0,1Q14,2013/4F,12.9,05/30/2014,...,-0.066202,0.488647,-0.227363,-0.197812,-0.11,30.82,-2.33,-0.016,-0.208,44.404971


In [18]:
dates = list(toy['report_date'])

In [19]:
dates[0]

'05/15/2014'

In [26]:
dt = datetime.strptime(dates[0], '%m/%d/%Y')
str(dt)

'2014-05-15 00:00:00'

In [27]:
new_dates = []

for d in dates:
    dt = datetime.strptime(d, '%m/%d/%Y')
    reformatted = str(dt)
    reformatted = reformatted[:10]
    new_dates.append(reformatted)





In [28]:
new_dates

['2014-05-15',
 '2014-04-25',
 '2014-04-04',
 '2014-05-02',
 '2014-05-01',
 '2014-04-25',
 '2014-05-02',
 '2014-04-24',
 '2014-04-23',
 '2014-05-30',
 '2014-05-09',
 '2014-05-07',
 '2014-05-13',
 '2014-05-07',
 '2014-06-03',
 '2014-05-01',
 '2014-04-30',
 '2014-05-02',
 '2014-04-29',
 '2014-05-14',
 '2014-05-07',
 '2014-04-30',
 '2014-04-24',
 '2014-05-07',
 '2014-04-23']